# 0. Data

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [41]:
'split data'
'first 21 columns are input variables and last 7 columns are output variables'

train_data = pd.read_csv('./data/SARCOSTst.csv', header=None)
test_data = pd.read_csv('./data/SARCOSTrn.csv', header=None)

X_train, X_valid, Y_train, Y_valid = train_test_split(train_data.iloc[:,:21], train_data.iloc[:,-7:], test_size=0.2)
X_test, Y_test = test_data.iloc[:,:21], test_data.iloc[:,-7:]

print('train set: ', X_train.shape, Y_train.shape)
print('valid set: ', X_valid.shape, Y_valid.shape)
print('test set: ', X_test.shape, Y_test.shape)

train set:  (31587, 21) (31587, 7)
valid set:  (7897, 21) (7897, 7)
test set:  (5000, 21) (5000, 7)


# 1. Nonlinear GP regression
- Gaussian kernel

# 2.Linear GP regression 
- Standard dot product kernel
    - k(a,b) = a^\top b